In [ ]:
# Install the below dependencies
import pandas as pd # dependency 1
from statsmodels.tsa.seasonal import seasonal_decompose # dependency 2
import matplotlib.pyplot as plt # dependency 3
import matplotlib.dates as mdates # dependency 4
from statsmodels.tsa.stattools import adfuller # dependency 5
from pmdarima.arima.utils import ndiffs # dependency 6
import seaborn as sns # dependency 7
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf # dependency 8

# Ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")

## Load the footfall dataset

In [ ]:
# Enter input paths for footfall data
path_footfall = 'C:/Users/medira/OneDrive - University of Leeds/Projects/AmbPop/AmbPopData/LCC_footfall_cleaned_2021.csv'

# Enter output paths for the folder storing outputs
dir_out = '../AmbPop_Outputs/'

In [ ]:
# dependency 1
# Load all footfall data to pandas DataFrame
df_orig = pd.read_csv(path_footfall)

df_orig.info()
df_orig.head(3)

## Ready the data for processing

In [ ]:
# Set dataframe index to 'DateTime' column
df_orig.set_index('DateTime', inplace=True)
df_orig.index = pd.to_datetime(df_orig.index)

df_orig.info()
df_orig.head(3)

In [ ]:
# Function to: create columns for TS EDA
def set_cols(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df['Year'] = df.Date.dt.year
    df['Month'] = df.Date.dt.month
    df['Hour'] = df.Date.dt.hour
    df['YearMonth'] = df.Date.dt.to_period('M')
    df['MonthDay'] = df.Date.dt.day
    df['WeekDay'] = df.Date.dt.weekday
    df['WeekDayName'] = df.Date.dt.day_name()
    df['IsWeekend'] = df.WeekDay > 4
    df['YearMonthDay'] = df['Year'].astype(str) + '-' + df['MonthDay'].astype(str)
    df['YearWeekDay'] = df['Year'].astype(str) + '-' + df['WeekDay'].astype(str)
    df['YearMonthWeekDay'] = df['YearMonth'].astype(str) + '-' + df['WeekDay'].astype(str)
    df['LocationYear'] = df['Location'].astype(str) + '-' + df['Year'].astype(str)
    df['LocationYearMonth'] = df['Location'].astype(str) + '-' + df['YearMonth'].astype(str)
    df['NumRecords'] =  df.groupby('Location')['Location'].transform('count')
    df['TotCountByLocationByYear'] = ( df.groupby(['Location', 'Year'])['Count'].transform('sum'))
    df['TotCountByWeekDayByYear'] = ( df.groupby(['WeekDay', 'Year'])['Count'].transform('sum'))
    df['TotCountByMonth'] = (df.groupby(['Month'])['Count'].transform('sum'))
        
    cols = ['Location', 'Count', 'LocationYear', 'LocationYearMonth', 'Date', 'Year', 'YearMonth', 
            'Month','YearMonthDay', 'MonthDay', 'YearWeekDay', 'WeekDay', 'YearMonthWeekDay',
            'WeekDayName', 'IsWeekend', 'BRCWeekNum', 'BRCMonthNum', 'BRCMonth', 'BRCYear', 
            'NumRecords', 'TotCountByLocationByYear', 'TotCountByWeekDayByYear', 'TotCountByMonth']
    return df[cols]

In [ ]:
# Filter the dataframe via predefined set_cols function to create all columns for EDA
df_all = set_cols(df_orig)

df_all.info()
df_all.head(3)

In [ ]:
# Remove missing values
df_all.dropna(inplace=True)

df_all.info()
df_all.head(3)

In [ ]:
# Keep locations with only full set of footfall records
df_all = df_all[df_all['NumRecords'] == 104904]

df_all.info()
df_all.head(3)

In [ ]:
# Sort the index
df_all.sort_index(inplace=True)

df_all.info()
df_all.head(3)

In [ ]:
# Create from existing, a dataframe with precovid timeline
df_precovid = df_all.loc[:pd.Timestamp('2020-01-01')]

df_precovid.info()
df_precovid.head(3)

In [ ]:
# Create from existing, a dataframe with covid timeline
df_covid = df_all.loc[pd.Timestamp('2020-01-01'):]

df_covid.info()
df_covid.head(3)

In [ ]:
# Function to: create dictionary of the timeseries split by location with location as key
def create_dict(df):
    loc_list = df['Location'].unique()
    dict = {loc : pd.DataFrame() for loc in loc_list}

    for key in dict.keys():
        dict[key] = df[:][df['Location'] == key]
        dict[key][['Count']]
    return dict

In [ ]:
# Pick a random location for TSA
loc_name = 'Headrow'

In [ ]:
# Get timeseries of the location from each of the dictionaries

In [ ]:
ts_loc = dict_all[loc_name]

ts_loc.info()
ts_loc.head(3)

In [ ]:
ts_loc_covid = dict_covid[loc_name]

ts_loc_covid.info()
ts_loc_covid.head(3)

In [ ]:
ts_loc_precovid = dict_precovid[loc_name]

ts_loc_precovid.info()
ts_loc_precovid.head(3)

In [ ]:
# Create three dictionaries with the three timeseries (all, covid, precovid)
dict_all = create_dict(df_all)
dict_covid = create_dict(df_covid)
dict_precovid = create_dict(df_precovid)

## EDA/Visualisation

In [ ]:
# Generate stats based on location
df_all.groupby('Location').describe()

In [ ]:
df_all[df_all.Count == df_all.Count.max()].iloc[0,0:2]

In [ ]:
df_all[df_all.Count == df_all.Count.min()].iloc[:,0:2]

In [ ]:
df_all[df_all.TotCountByLocationByYear == df_all.TotCountByLocationByYear.max()].iloc[0,0:2]

In [ ]:
plt.rcParams["figure.figsize"] = (15,10)
fig = plt.figure()
ax = fig.add_subplot()
df_all_samp1 = df_all.drop_duplicates('LocationYear')
for name,group in df_all_samp1.groupby(['Year']):
    group.plot.line(ax=ax, x="Location",y="TotCountByLocationByYear", label= str(name))
    ax.set_ylabel('Footfall Count By Year')
    ax.set_xlabel('Location')
    plt.title('Total Count by Location per Year', fontsize=15)
    ax.tick_params(axis='x', rotation=45)

plt.show()

In [ ]:
df_all_samp2 = df_all.drop_duplicates('YearWeekDay')
print(df_all_samp2.groupby(['WeekDay']).count())

In [ ]:
df_all_samp2 = df_all_samp2[['WeekDayName', 'Year', 'TotCountByWeekDayByYear']]
df_all_samp2 = df_all_samp2.pivot(index='WeekDayName', columns='Year')

df_all_samp2.columns = df_all_samp2.columns.droplevel(0)
df_all_samp2.info()
df_all_samp2.head(3)

In [ ]:
plt.rcParams["figure.figsize"] = (15,10)
fig = plt.figure()
ax = fig.add_subplot()
df_all_samp2.plot(ax=ax, kind='bar')
ax.set_ylabel('all Count By Year')
ax.set_xlabel('Week Day')
plt.title('Total Count per Week Day per Year', fontsize=15)
ax.tick_params(axis='x', rotation=45)

plt.show()

In [ ]:
df_all_samp4 = df_all.drop_duplicates('Month')
df_all_samp4.groupby(['Month']).count().iloc[0,0:2]
df_all_samp4 = df_all_samp4[['Month', 'TotCountByMonth']]

df_all_samp4.info()
df_all_samp4.head(12)

## TSA

In [ ]:
# Dependency 2
# display seasonal decompose result of all the locations
for key in dict_all.keys():
    result = seasonal_decompose(dict_all[key]['Count'], period=1)
    print('Location: ' + key)
    fig = plt.figure()  
    fig = result.plot()  
    fig.set_size_inches(12, 9)
    plt.show()

In [ ]:
# Dependency 3
# Dependency 4
# nf
# Function to: plot the timeseries and stats with adjustable datetime x axis, rolling stats sections 
def plot_df(ts, name, interval, sec, choice):
    fig, ax = plt.subplots(figsize=(15,7))
    ax.plot(ts.index, ts.Count, label='Original')
    
    if 'rolmean' in choice:
        rolmean = ts['Count'].rolling(sec).mean()
        ax.plot(rolmean, label='Rolling Mean')
    if 'rolstd' in choice:
        rolstd = ts['Count'].rolling(sec).std()
        ax.plot(rolstd, label = 'Rolling Std')
    if 'expmean' in choice:
        expmean = ts['Count'].ewm(sec).mean()
        ax.plot(expmean, label = 'Exp Mean')
    
    plt.title(name, fontsize=15)
    plt.legend(loc='best')
    ax.set_ylabel('Footfall Count')
    ax.set_xlabel('DateTime')
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=interval))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%d-%m-%Y'))
    plt.gcf().autofmt_xdate()
    plt.show()
    

In [ ]:
# nf
# Dependency 5
# Display the Dickey-Fuller test for stationarity of the timeseries
def test_stationarity(ts):
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(ts['Count'])
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

In [ ]:
# nf
# Plot the timeseries of the selected location
plot_df(ts_loc, loc_name, 12, 12, ['rolmean', 'rolstd', 'expmean'])

In [ ]:
# Test the stationarity of the timeseries
test_stationarity(ts_loc)

In [ ]:
# Dependency 6
# nf
# Adf test
#ndiffs(ts_loc, test='adf')  #0

# KPSS test
#ndiffs(ts_loc, test='kpss')  #1

# PP test:
#ndiffs(ts_loc, test='pp') #0

In [ ]:
# Dependency 7
# nf
# Display the trend and seasonality via boxplot
fig, axes = plt.subplots(1, 2, figsize=(15,7))
sns.boxplot(x='Year', y='Count', data=ts_loc, ax=axes[0])
sns.boxplot(x='Month', y='Count', data=ts_loc, ax=axes[1])

axes[0].set_title(loc_name + ' Year-wise Box Plot - (The Trend) '); 
axes[1].set_title(loc_name + ' Month-wise Box Plot - (The Seasonality) ')
plt.show()

In [ ]:
# Dependency 8
# nf

# Original timeseries
fig, axes = plt.subplots(3, 3)
axes[0, 0].plot(ts_loc); axes[0, 0].set_title('Original Series')
plot_acf(ts_loc, ax=axes[0, 1], lags=20)
plot_pacf(ts_loc, ax=axes[0, 2], lags=20)

# 1st differencing
axes[1, 0].plot(ts_loc.diff()); axes[1, 0].set_title('1st Order Differencing')
plot_acf(ts_loc.diff().dropna(), ax=axes[1, 1], lags=20)
plot_pacf(ts_loc.diff().dropna(), ax=axes[1, 2], lags=20)


# 2nd differencing
axes[2, 0].plot(ts_loc.diff().diff()); axes[2, 0].set_title('2nd Order Differencing')
plot_acf(ts_loc.diff().diff().dropna(), ax=axes[2, 1], lags=20)
plot_pacf(ts_loc.diff().diff().dropna(), ax=axes[2, 2], lags=20)

plt.show()